<a href="https://colab.research.google.com/github/elisa641995/po/blob/main/skin_care.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
import re
import time
import nltk
import torch
from typing import Dict, List, Optional, Union, Tuple, Callable
import math
import torch.nn as nn
import torch.nn.functional as F
import timeit
import string
import random
import zipfile
import datetime
import textwrap
#import torchtext
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import matplotlib as mpl
import scipy.stats as st
import missingno as msno
from tqdm.auto import tqdm
from rich.text import Text
import plotly.express as px
from tensorflow import keras
from bs4 import BeautifulSoup
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
from rich.console import Console
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from transformers import pipeline
import matplotlib.ticker as ticker
import tensorflow_datasets as tfds
from keras.models import Sequential
#from torchtext import data, datasets
from keras.layers import Dense, Dropout
from IPython.display import display, HTML
#from data_profiling import ProfileReport
from wordcloud import WordCloud, STOPWORDS
from matplotlib.ticker import NullFormatter
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML, display, Markdown
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
from tensorflow.keras.layers import Embedding, SpatialDropout1D, Bidirectional, LSTM, Dense, Dropout
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from transformers import DistilBertForSequenceClassification, AdamW, DistilBertTokenizer, BertTokenizer, BertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import f1_score
nltk.download('stopwords')
sns.set(color_codes = True)
sns.set_palette(palette = 'RdGy', n_colors = 8)
colors = ["#C20000", "#D65F5F", "#EBA3A3", "#F3C1C1", "#E1E1E1", "#B0B0B0", "#7F7F7F", "#4C4C4C"]
sns.palplot(sns.color_palette(colors))
sns.set(style="whitegrid")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


##Exploratory data analysis of the product data


In [ ]:
data_product.head()

In [ ]:
#Precentage of null values in every column
data_product.isnull().sum()*100/data_product.shape[0]

I recommend dropping the following columns from the dataset, as they contain a significant amount of missing values: `child_max_price`, `child_min_price`, `value_price_usd`, `sale_price_usd`, and `variation_desc`.

In [ ]:
data_product.shape

In this project, I will focus specifically on skincare products from Sephora. To ensure accurate analysis and insights, I will separate the skincare-related data from other categories, such as makeup. This approach will allow for a more precise evaluation of trends and performance within the skincare segment.

In [ ]:
string_to_check =['Remover' , 'remover' , 'Cleanser' , 'cleanser' , 'Make up cleanser' , 'bb cream' , 'bbcream' , 'BB' , 'Mask' , 'Masks' , 'mask' , 'Lip balm' , 'lip balm' , 'balm' , 'Gel' , 'gel' , 'make up remover' , 'Make up remover' ,'Hairdresser', 'Curl','Repair','Thickening',"Hairdresser",'Heat' ,'shampoo','Shampoo','hair','haircare','Haircare','Style','Styler','style','scalp','Scalp','Conditioner','conditioner','Frizz']
data_product = data_product[data_product['product_name'].str.contains('|'.join(string_to_check))]
data_product.reset_index()

In [ ]:
data_product.shape

In [ ]:
#checking for duplicates in the data
data_product.duplicated().sum()

In [ ]:
#checking data type of the columns
print(data_product.dtypes)

#Let’s begin uncovering the story the data has to share with us.

In [ ]:
import pandas as pd

data = [['John', 50, 'Male', 'Austin', 70],
        ['Cataline', 45 ,'Female', 'San Francisco', 80],
        ['Matt', 30 ,'Male','Boston', 95]]

# Column labels of the DataFrame
columns = ['Name','Age','Sex', 'City', 'Marks']

# Create a DataFrame df
df = pd.DataFrame(data, columns=columns)

df['Sex'].value_counts()

In [ ]:
columns_to_plot = [ 'rating', 'price_usd', 'child_max_price', 'child_min_price']
plt.figure(figsize=(15, 10))
for i, column in enumerate(columns_to_plot, 1):
    plt.subplot(2, 3, i)
    plt.hist(data_product[column].dropna(), bins=30, color='blue', alpha=0.7)
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


As we can see, the majority of the products have a rating above 3.5, indicating that customers are generally satisfied with their purchases. Additionally, the prices for skincare products typically fall within a range of up to $80, with only a few products exceeding that price point.

In [ ]:
#checkin if there is multucoluniarity in the data using VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
columns_to_multi = [ 'rating', 'price_usd', 'sale_price_usd', 'child_max_price', 'child_min_price','loves_count','price_usd','sale_price_usd','child_max_price','child_min_price']
X = data_product[columns_to_multi].dropna()

# Calculate VIF for each column
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

Examining the correlations between the columns

In [ ]:
columns_to_multi = [ 'rating', 'price_usd', 'sale_price_usd', 'child_max_price', 'child_min_price','loves_count','price_usd','sale_price_usd','child_max_price','child_min_price']
df_selected = data_product[columns_to_multi].dropna()
corr_matrix = df_selected.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()


It is evident that there are significant correlations between various variables related to product pricing in the data. For instance, there's a strong positive correlation between `min_child_price` and `sale_price_usd`. This suggests that as the original product's price increases, the minimum price of the child product also tends to rise.

Investigating Product Categories

In [ ]:
data_product['primary_category'].unique()

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(data=data_product, x='rating', hue='primary_category', common_norm=False, fill=True, alpha=0.5)
plt.title('Distribution of Ratings by Primary Category (KDE)')
plt.xlabel('Rating')
plt.ylabel('Density')
plt.show()

The distribution varies by category, but overall, the majority of ratings seem to cluster around 4 and 5, indicating that products in these categories generally receive high ratings.Categoris Fragrance, Hair & Men This category have a relatively narrow distribution, peaking around 4.5-5. Its density is higher, indicating that most of the ratings are clustered around the high end (4.5 to 5), with few ratings below 4.

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(data=data_product, x='price_usd', hue='primary_category', common_norm=False, fill=True, alpha=0.5, bw_adjust=0.5)
plt.xlim(0, 350)
plt.title('Distribution of price $ by Primary Category (KDE)')
plt.xlabel('Price (USD)')
plt.ylabel('Density')
plt.show()


Fragrance and skincare products exhibit a broader price distribution compared to most other categories. While both peak below 50, fragrance products show a long tail that extends beyond 250, reflecting a wide price range, including many high-end items. Hair, makeup, and men’s products, on the other hand, have sharper peaks around 50 and 30, respectively, but their distributions are much narrower, rapidly tapering off after $50. This suggests that the majority of products in these categories are concentrated in the lower price range, typically under 50 .

Mini Size products stand out with the sharpest peak, around 10, and the narrowest distribution. This indicates that these items are tightly priced, mostly under $20, with very few products priced higher.

In [ ]:
columns_to_plot=['child_min_price','sale_price_usd','child_max_price']
fig = plt.figure(figsize=(25, 20))
for i,col in enumerate(columns_to_plot):
  ax = fig.add_subplot(5, 3, i + 1)
  sns.kdeplot(data=data_product, x=col, hue='primary_category', common_norm=False, fill=True, alpha=0.5)
  ax.set_title(f'Distribution of {col}  by Primary Category (KDE)')
  ax.set_xlabel(f'{col}')
  ax.set_ylabel('Density')
plt.tight_layout()
plt.show()

As noted earlier, the fragrance category shows a broader price distribution for its subcategories, covering a wide range of minimum and maximum prices. In contrast, skincare products exhibit a narrower distribution among their subcategories, while hair products display a wider spread. Mini size, makeup, and bath products all have a more concentrated price range, with peaks around 20-30 dollars, indicating that most of the subcategory products fall within this price bracket.

In [ ]:
#measure skewnes of dara
data_product[columns_to_plot].skew(axis = 0, skipna = True)

Looking for some outlires

In [ ]:
columns_to_plot_box=['rating','price_usd','sale_price_usd','child_min_price','child_max_price']
for i,col in enumerate(columns_to_plot_box):
    plt.figure(figsize=(15, 3))
    sns.boxplot(x=data_product[col])
    quantiles = np.quantile(data_product[col].dropna(), [0.25, 0.5, 0.75])
    mean = data_product[col].dropna().mean()
    title = f"Distribution of {col}\n"
    title += f"Q1: {quantiles[0]:.2f}, Median: {quantiles[1]:.2f}, Q3: {quantiles[2]:.2f}, Mean: {mean:.2f}"
    plt.title(title)
    plt.xlabel(f'{col}')
    plt.show()

When examining the box plot, we notice several data points that might initially appear as outliers. However, considering the context—consumer reviews and sale prices—these points may not truly be outliers. Consumer opinions can vary widely, as each individual has unique preferences, meaning their reviews shouldn’t be dismissed as anomalies. Similarly, high sale prices for particularly successful or premium products, like those in the fragrance category, shouldn’t be seen as outliers either. As we’ve observed, fragrance products often have a broader price range, which can lead to the "outliers" displayed on the box plot. One exception is a product priced above 1,750, which belongs to the skincare brand Shani Darden. After a quick check on their website, their products typically range up to $200, suggesting this is likely a typo.

In [ ]:
num_limited_products_primery_category = data_product.groupby('primary_category')['limited_edition'].sum()
plt.figure(figsize=(12, 6))
sns.barplot(x=num_limited_products_primery_category.index, y=num_limited_products_primery_category.values)
plt.title('Number of Limited Edition Products by Primary Category')
plt.xlabel('Primary Category')
plt.ylabel('Number of Limited Edition Products')
plt.show()

We can observe that limited edition products are more commonly found in the primary categories of hair, makeup, and skincare. Now, let's explore whether there is a relationship between a product being labeled as a limited edition and its price.

In [ ]:
import scipy.stats as stats
x=data_product['limited_edition']
y=data_product['price_usd']
stats.pointbiserialr(x, y)



As we can see, there is no significant correlation between a product being categorized as a limited edition and its price.

In [ ]:
#Let's take a closer look at the top-rated and most expensive brands in the skincare.
skin_brands = data_product[(data_product['primary_category'] == 'Skincare')&(data_product['brand_id'] !=6314)]
expensive_best_rating_brands = skin_brands.groupby('brand_name').agg({'price_usd': 'median', 'rating': 'median'})
top_brands = expensive_best_rating_brands.sort_values(by=['price_usd','rating'], ascending=False).head(10)
top_brands.plot(kind='bar', figsize=(10, 6))
plt.title('Top ten brands by median price and rating')
plt.xlabel('brands')
plt.ylabel('Median Value')
plt.xticks(rotation=45,fontsize=8)
plt.show()


##Reviews data

In [ ]:
df_1=pd.read_csv('/content/reviews_0-250.csv')
df_2=pd.read_csv('/content/reviews_1250-end.csv')
df_3=pd.read_csv('/content/reviews_250-500.csv')
df_4=pd.read_csv('/content/reviews_500-750.csv')
df_5=pd.read_csv('/content/reviews_750-1250.csv')


In [ ]:
df_1.head()

In [ ]:
#lets look at the shapes of all the reviews data frames
list_of_dfs=[df_1,df_2,df_3,df_4,df_5]
for i in list_of_dfs:
  print(i.shape)

In [ ]:
#combine the all the data frames one under the other
all_reviews=pd.concat(list_of_dfs,axis=0)

In [ ]:
#checking the concatination was done corectly
num_rows=0
for i in list_of_dfs:
  num_rows+=i.shape[0]
print(num_rows)

In [ ]:
print(all_reviews.shape)

In [ ]:
#Let's examine the percentage of NaN values in each column of the all_reviews dataset.
all_reviews.isnull().sum() * 100 / len(all_reviews)

In [ ]:
all_reviews['rating'].unique()

In [ ]:
#quick checking some reviews of ratign 3 like 20 examples  , kust the review text make google colab present the whole sentence

# Adjust display settings to show the full text
pd.set_option('display.max_colwidth', None)

# Now display the reviews
all_reviews[all_reviews['rating'] == 3][['review_text','rating']].sample(20)


When performing sentiment analysis to classify feedback into two categories (good/bad), or even three, I found that comments with a 3-star rating often tell an interesting story. After reviewing these reviews, I, as a potential buyer, would likely avoid purchasing these products. Many of the comments suggest that the products don't necessarily solve the issue, or that similar quality can be found at a lower price. While these products aren't considered bad, the level of satisfaction expressed isn't strong enough to convince me to buy them. When using sentiment analysis to predict whether customers will repurchase a product or if it's worth continuing production, items with a 3-star rating wouldn't fall into the "I'm buying it" category.There for I decided to have just 2 categories - good/bad

In [ ]:
#sentiment column
all_reviews['sentiment'] = all_reviews['rating'].apply(lambda x: 0 if x <= 3 else 1)

In [ ]:
all_reviews['sentiment_text']=all_reviews['sentiment'].map({0:'negative',1:'positive'})

In [ ]:
all_reviews = all_reviews.dropna(subset=['review_text'])

In [ ]:
#Percentage breakdown of positive, negative reviews.
all_reviews['sentiment_text'].value_counts()/len(all_reviews)*100

We can observe that our dataset is imbalanced, as indicated by the product data, which contains more positive ratings than negative ones

In [ ]:
#checking duplicates in the data
print('Number of duplicated rows: ' , len(all_reviews[all_reviews.duplicated()]))

In [ ]:
data_product['primary_category'].unique()

In [ ]:
all_reviews = pd.merge(all_reviews, data_product[['product_id', 'secondary_category']], on='product_id', how='left')

In [ ]:
#"Let's examine the percentage of positive and negative ratings within each primary category.
all_reviews.groupby('secondary_category')['sentiment_text'].value_counts(normalize=True) * 100

We can observe that there is class imbalance across all secondary category products.

Due to resource limitations, specifically in GPUs and memory, I will fine-tune the RoBERTa model only for the Moisturizers and Cleansers categories

In [ ]:
interested_categories=['Moisturizers','Cleansers']

In [ ]:
all_reviews_2 = all_reviews[all_reviews['secondary_category'].isin(interested_categories)]

In [ ]:
all_reviews_2.shape

We can conclude that, overall, customers are satisfied with the products, as reflected in their use of positive words such as 'love,' 'effective,' 'worth,' 'good,' 'holy grail,' 'must-have,' and 'best.' Additionally, it appears that customers are particularly interested in products designed to combat dry skin, such as moisturizers and hydrating formulations.

In [ ]:
review_text = all_reviews['review_text'].dropna()
text = ' '.join(review_text)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = STOPWORDS,
                min_font_size = 10).generate(text)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

Upon examining the detailed comments from customers, we notice that products addressing sensetive skin are frequently mentioned, and the topic of skin is one of the most popular. Additionally, the word 'love' appears quite often indicating product satisfaction

Lets look what words are mostly assosiated with good reviews

In [ ]:
from wordcloud import WordCloud, STOPWORDS
positive_reviews = all_reviews[all_reviews['sentiment'] == 1]
review_text_pos = positive_reviews['review_text']
text = ' '.join(review_text_pos)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = STOPWORDS,
                min_font_size = 10).generate(text)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

We can see that words such as 'love,' 'amazing,' 'recommend,' 'well,' 'great,' 'gentle,' and others are strongly associated with positive reviews

In [ ]:
bad_rev = all_reviews[all_reviews['sentiment'] == 0]
review_text = bad_rev['review_text']
text = ' '.join(review_text)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = STOPWORDS,
                min_font_size = 10).generate(text)
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

We can observe that words such as 'price,' 'unfortunately,' 'redness,' and 'dry skin' are commonly associated with negative reviews.

# HTML Tags

In [ ]:
#checking for HTMLS
import re
def find_html(text):
    html_pattern = r'<[^>]+>'
    return re.findall(html_pattern, text)
html_in_reviews = all_reviews['review_text'].apply(find_html)
html_in_reviews = html_in_reviews[html_in_reviews.str.len() > 0]

#Upper case words

Let's examine whether there are any words that are written entirely in uppercase. In sentiment analysis, such words can often indicate strong emotions.

In [ ]:
from itertools import chain

def is_upper_case(text):
  return [word for word in text.split() if word.isupper() and word != 'I']

unique_upper_words = set(chain.from_iterable(all_reviews['review_text'].apply(is_upper_case)))


#URLS


In [ ]:
#checking if there are URLS in the data
def find_urls(text):
    url_pattern = r'(https?://\S+|www\.\S+)'
    return re.findall(url_pattern, text)
urls_in_reviews = all_reviews['review_text'].apply(find_urls)
urls_in_reviews = urls_in_reviews[urls_in_reviews.str.len() > 0]


In [ ]:
sentence = "Check out this website: https://www.example.com and this one: http://anothersite.net."

tokens = tokenizer.tokenize(sentence)
print(tokens)

original = tokenizer.convert_tokens_to_string(tokens)
print(original)

#Tags

In [ ]:
def find_at_symbols(text):
    at_pattern = r'@'
    return re.findall(at_pattern, text)
at_symbols_in_reviews = all_reviews['review_text'].apply(find_at_symbols)
at_symbols_in_reviews = at_symbols_in_reviews[at_symbols_in_reviews.str.len() > 0]


In [ ]:
sentence = "Hey @username, check this out! Also, @anotheruser might be interested."

tokens = tokenizer.tokenize(sentence)
print(tokens)

original = tokenizer.convert_tokens_to_string(tokens)
print(original)

# checking for emojis

In [ ]:
def find_emojis(text):
    emoji_pattern = re.compile(
        r'['
        u'\U0001F600-\U0001F64F'  # Emoticons
        u'\U0001F300-\U0001F5FF'  # Symbols & Pictographs
        u'\U0001F680-\U0001F6FF'  # Transport & Map Symbols
        u'\U0001F700-\U0001F77F'  # Alchemical Symbols
        u'\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
        u'\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
        u'\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
        u'\U0001FA00-\U0001FA6F'  # Chess Symbols
        u'\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
        u'\U00002702-\U000027B0'  # Dingbats
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    return emoji_pattern.findall(text)
emojis_in_reviews = all_reviews['review_text'].apply(find_emojis)
emojis_in_reviews = emojis_in_reviews[emojis_in_reviews.str.len() > 0]

Testing if RoBERTA model can tokenize the emojies

In [ ]:
from transformers import RobertaModel, RobertaTokenizer

In [ ]:

# Load the tokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=False)

In [ ]:
a=tokenizer.tokenize("Mixed feelings about this 🤔🤷‍♂️")
print(a)

In [ ]:
original_sentence = tokenizer.convert_tokens_to_string(a)
print(original_sentence)

In [ ]:
#LETS TRY ANOTHER SENTENCE WITH HEART TOKENS WHICH ARE RELEVANT FOR THIS TOPIC
b=tokenizer.tokenize("I love this product ❤️❤️❤️❤️❤️❤️❤️")
print(b)

In [ ]:
#now lets decode it back to the original sentence
original_sentence = tokenizer.convert_tokens_to_string(b)
print(original_sentence)

The tokenizer performs well, effectively recognizing and tokenizing HTML elements, URLs, emojis, and tags.✌

In [ ]:
#does it tokenize tags like @username?
c=tokenizer.tokenize("@username used this")
print(c)
original_sentence = tokenizer.convert_tokens_to_string(c)
print(original_sentence)

In [ ]:
#and does it tokenize URL?
d=tokenizer.tokenize("Check out this website: https://www.example.com and this one: http://anothersite.net.")
print(d)
original_sentence = tokenizer.convert_tokens_to_string(d)
print(original_sentence)

In [ ]:
#and does it tokenize HTML ?
e=tokenizer.tokenize("This is a sentence with some <b>bold</b> text and a <a href='https://www.example.com'>link</a>.")
print(e)
original_sentence = tokenizer.convert_tokens_to_string(e)
print(original_sentence)

so far looks good!

## Fine tuning RoBERTA

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
device

In [ ]:
new_df=all_reviews_2[['review_text','sentiment']]

In [ ]:
new_df.shape

In [ ]:
new_df.head()

#Loading the data

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=False)


In [ ]:
#=precentage the number of posotove negative and neutral reviews
new_df['sentiment'].value_counts()/len(new_df)*100

In [ ]:
new_df['sentiment'].value_counts()

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.review_text
        self.targets = self.data.sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),#1 if masked 0 if not
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)#The sentiment label for this text.
        }

In [ ]:
from sklearn.model_selection import train_test_split

# First, split into train+validation and test sets
train_val_size = 0.8
train_val_data = new_df.sample(frac=train_val_size, random_state=200)
test_data = new_df.drop(train_val_data.index).reset_index(drop=True)

# Now split train_val_data into train and validation sets
# Validation set will be 10% of train_val_data, which is 12.5% of the original train set (0.1 / 0.8 = 0.125)
train_data, val_data = train_test_split(train_val_data, test_size=0.125, random_state=200)

# Reset indices
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("VALIDATION Dataset: {}".format(val_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
validation_set = SentimentData(val_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
Validation_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }
training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
validation_loader = DataLoader(validation_set, **Validation_params)

#Creating the Neural Network for Fine Tuning

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

In [ ]:
count_clases=new_df['sentiment'].value_counts()
count_clases

In [ ]:
class_weights = np.array(count_clases/np.sum(count_clases))
class_weights

In [ ]:
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

#Training

In [ ]:
#After every 2500 steps the loss value is printed in the console.

In [ ]:
from sklearn.metrics import f1_score
import os
def train_and_validate(epochs, model, training_loader, validation_loader, loss_function, optimizer, device, patience=2):
    model_save_path = '/content/drive/MyDrive/best_model.pt'
    os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Training
        model.train()
        tr_loss = 0
        all_tr_targets = []
        all_tr_outputs = []
        nb_tr_steps = 0

        for _, data in tqdm(enumerate(training_loader, 0), desc=f"Training Epoch {epoch+1}"):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()

            big_val, big_idx = torch.max(outputs.data, dim=1)
            all_tr_targets.extend(targets.cpu().detach().numpy())
            all_tr_outputs.extend(big_idx.cpu().detach().numpy())
            nb_tr_steps += 1
            if _ % 2500 == 0:
                loss_step = tr_loss / nb_tr_steps
                print(f"Training Loss per 2500 steps: {loss_step}")

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()



        epoch_loss = tr_loss / nb_tr_steps
        train_losses.append(epoch_loss)

        tr_f1_scores = f1_score(all_tr_targets, all_tr_outputs, average=None)
        tr_macro_f1 = np.mean(tr_f1_scores)
        print(f"Training Loss Epoch {epoch+1}: {epoch_loss}")
        print(f"Training F1 Scores: {tr_f1_scores}")
        print(f"Training Macro F1 Score: {tr_macro_f1}")

        # Validation
        model.eval()
        val_loss = 0
        all_val_targets = []
        all_val_outputs = []
        nb_val_steps = 0

        with torch.no_grad():
            for _, data in tqdm(enumerate(validation_loader, 0), desc=f"Validation Epoch {epoch+1}"):
                ids = data['ids'].to(device, dtype=torch.long)
                mask = data['mask'].to(device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
                targets = data['targets'].to(device, dtype=torch.long)

                outputs = model(ids, mask, token_type_ids)
                loss = loss_function(outputs, targets)
                val_loss += loss.item()

                big_val, big_idx = torch.max(outputs.data, dim=1)
                all_val_targets.extend(targets.cpu().detach().numpy())
                all_val_outputs.extend(big_idx.cpu().detach().numpy())

                nb_val_steps += 1

        val_epoch_loss = val_loss / nb_val_steps
        val_losses.append(val_epoch_loss)

        val_f1_scores = f1_score(all_val_targets, all_val_outputs, average=None)
        val_macro_f1 = np.mean(val_f1_scores)
        print(f"Validation Loss Epoch {epoch+1}: {val_epoch_loss}")
        print(f"Validation F1 Scores: {val_f1_scores}")
        print(f"Validation Macro F1 Score: {val_macro_f1}")

        # Early stopping logic
        if val_epoch_loss < best_val_loss:
            best_val_loss = val_epoch_loss
            patience_counter = 0  # Reset the patience counter if loss improves
            torch.save(model.state_dict(), model_save_path)  # Save best model to Google Drive
            print(f"Best model saved with Validation Loss: {best_val_loss}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

    # Plot training and validation losses
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label="Training Loss")
    plt.plot(range(1, len(val_losses) + 1), val_losses, label="Validation Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.legend()
    plt.grid()
    plt.show()

    print("Training and Validation completed!")

# Usage
epochs = 5
train_and_validate(epochs, model, training_loader, validation_loader, loss_function, optimizer, device)


The early stopping rule was triggered at epoch 3, indicating that further training would likely result in overfitting. Additionally, the validation loss is lower than the training loss, which may be due to the validation samples being drawn from the training set, resulting in high similarity between the two.

In [ ]:
# Load the model
model.load_state_dict(torch.load('/content/drive/MyDrive/best_model.pt'))
model.to(device)


In [ ]:
#test data
def test_my_model(model, testing_loader,device):
    model.eval()
    all_test_targets = []
    all_test_outputs = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            big_val, big_idx = torch.max(outputs, dim=1)
            all_test_targets.extend(targets.cpu().detach().numpy())
            all_test_outputs.extend(big_idx.cpu().detach().numpy())
    test_f1_scores = f1_score(all_test_targets, all_test_outputs, average=None)
    test_macro_f1 = np.mean(test_f1_scores)
    print(f"test F1 Scores: {test_f1_scores}")
    print(f"test Macro F1 Score: {test_macro_f1}")
test_my_model(model, testing_loader,device)

After applying our model to the test dataset, we observed high F1 scores for both the positive and negative classes, indicating strong performance in balancing precision and recall. This means the model effectively identifies true positives while minimizing false positives and negatives. In our case, with an imbalanced dataset, the F1 score is especially valuable as it offers a more insightful evaluation than accuracy alone, showing the model’s ability to distinguish between sentiment classes with optimized precision and recall.